# Urban digital twin scenarios
For the spatial operations you need at least arcpy or shapely and pyproj installed.

In [1]:
from arcgis.features import FeatureSet, GeoAccessor
from codecarbon import track_emissions
from configparser import ConfigParser
import json
import logging
import os
import pandas as pd
import sqlite3 as sql
import warnings

In [2]:
def read_geojson_as_sdf(filepath: str, encoding: str='utf8'):
    """
    Reads a GeoJSON file as a FeatureSet.
    """
    with open(filepath, encoding=encoding) as in_stream:
        return FeatureSet.from_geojson(json.load(in_stream)).sdf
    
def read_sqlite_as_sdf(db_filepath: str, select_statement: str, x_column: str='longitude', y_column: str='latitude'):
    """
    Reads the data from a sqlite database into main memory using a SQL statement.
    """
    with sql.connect(db_filepath) as connection:
        df = pd.read_sql_query(select_statement, connection)
        return GeoAccessor.from_xy(df, x_column, y_column)

In [3]:
config = ConfigParser()
config.read('config.user')

['config.user']

In [5]:
@track_emissions(project_name='Urban Digital Twin Bonn', output_file='emissions.user', offline=True, country_iso_code='USA')
def urban_intersect():
    logger = logging.getLogger('codecarbon')
    logger.info('Load urban datasets...')
    districts_sdf = read_geojson_as_sdf(config['DEFAULT']['CityDistrictsFilePath'], encoding='cp1252')
    streets_sdf = read_geojson_as_sdf(config['DEFAULT']['CityStreetFilePath'])
    if 'area' in streets_sdf.columns:
        streets_sdf = streets_sdf[streets_sdf['area'].isna()]
    traffic_sdf = read_sqlite_as_sdf(config['DEFAULT']['TrafficFilePath'], 'SELECT * from agent_pos LIMIT 10;')
    logger.info('Urban datasets loaded.')
    
    logger.info('Intersecting traffic with city districts...')
    traffic_joined_districts_sdf = traffic_sdf.spatial.join(districts_sdf, how='inner', op='intersects')
    logger.info(f'{traffic_joined_districts_sdf.shape[0]} traffic locations have intersections with city districts.')
    
    warnings.filterwarnings('ignore')
    
    logger.info('Projecting city streets...')
    streets_sdf.spatial.project(25832, 'DHDN_To_WGS_1984_4_NTv2')
    logger.info(f'{streets_sdf.shape[0]} city streets were projected.')
    
    logger.info('Projecting traffic locations...')
    traffic_sdf.spatial.project(25832, 'DHDN_To_WGS_1984_4_NTv2')
    logger.info(f'{traffic_sdf.shape[0]} traffic locations were projected.')
    
    warnings.filterwarnings('default')
    
    logger.info('Constructing city streets buffer...')
    buffered_streets_sdf = streets_sdf.copy()
    buffered_streets_sdf.SHAPE = streets_sdf.SHAPE.geom.buffer(50)
    logger.info(f'{buffered_streets_sdf.shape[0]} city streets buffer were constructed.')
    
    logger.info('Intersecting traffic with city streets buffer...')
    traffic_joined_buffered_streets_sdf = traffic_sdf.spatial.join(buffered_streets_sdf, how='inner', op='intersects')
    logger.info(f'{traffic_joined_districts_sdf.shape[0]} traffic locations have intersections with city streets buffer.')
    
    # Join traffic with intersected streets buffer with original streets
    # Calculate distance between traffic location and corresponding streets
    logger.info('Calculating distances to city streets...')
    traffic_joined_streets_sdf = pd.merge(traffic_joined_buffered_streets_sdf, streets_sdf, how='left', on='F_id', suffixes=['', '_street'])
    traffic_joined_streets_sdf['distance_to'] = traffic_joined_streets_sdf.SHAPE.geom.distance_to(traffic_joined_streets_sdf.SHAPE_street)
    logger.info(f'{traffic_joined_streets_sdf.shape[0]} distances to city streets were calculated.') 

In [6]:
urban_intersect()

[codecarbon INFO @ 13:39:59] offline tracker init
[codecarbon INFO @ 13:39:59] [setup] RAM Tracking...
[codecarbon INFO @ 13:39:59] [setup] GPU Tracking...
[codecarbon INFO @ 13:40:00] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 13:40:00] [setup] CPU Tracking...
[codecarbon WARNING @ 13:40:00] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 13:40:02] We saw that you have a Intel(R) Core(TM) i7-7820HQ CPU @ 2.90GHz but we don't know it. Please contact us.
[codecarbon INFO @ 13:40:02] CPU Model on constant consumption mode: Intel(R) Core(TM) i7-7820HQ CPU @ 2.90GHz
[codecarbon INFO @ 13:40:02] >>> Tracker's metadata:
[codecarbon INFO @ 13:40:02]   Platform system: Windows-10-10.0.22621-SP0
[codecarbon INFO @ 13:40:02]   Python version: 3.9.18
[codecarbon INFO @ 13:40:02]   CodeCarbon version: 2.2.2
[codecarbon INFO @ 13:40:02]   Available RAM : 31.807 GB
[codecarbon INFO @ 13:40:02]   CPU count: 8
[codecarbon INFO @ 13:40:02]   CPU model: Intel(

AttributeError: 'list' object has no attribute 'values'

In [57]:

streets_sdf

,@id,area,bicycle,highway,name,surface,OBJECTID,SHAPE
28,way/22880467,<NA>,<NA>,pedestrian,Takuplatz,<NA>,29,"{""rings"": [[[6.9158428, 50.9615675], [6.915798..."
137,way/36809213,<NA>,<NA>,no,Ubierring,<NA>,138,"{""rings"": [[[6.9658678, 50.9225633], [6.965856..."
138,way/36809214,<NA>,<NA>,no,Ubierring,<NA>,139,"{""rings"": [[[6.9650702, 50.9222487], [6.965081..."
154,way/40291091,<NA>,<NA>,no,Ubierring,<NA>,155,"{""rings"": [[[6.9663071, 50.9218984], [6.966307..."
155,way/41362803,<NA>,<NA>,no,Chlodwigplatz,<NA>,156,"{""rings"": [[[6.9602065, 50.9212555], [6.960215..."
...,...,...,...,...,...,...,...,...
151045,way/557097695,<NA>,<NA>,service,<NA>,<NA>,151046,"{""paths"": [[[7.0551821, 50.9670198], [7.055306..."
151046,way/557154755,<NA>,<NA>,path,<NA>,unpaved,151047,"{""paths"": [[[6.9486374, 50.9557695], [6.949021..."
151047,way/557154756,<NA>,<NA>,path,<NA>,unpaved,151048,"{""paths"": [[[6.9489955, 50.9560589], [6.948642..."
151048,way/557154757,<NA>,<NA>,tertiary,Merheimer Straße,<NA>,151049,"{""paths"": [[[6.9496619, 50.9584811], [6.949690..."


In [77]:
import shapely

ModuleNotFoundError: No module named 'shapely'